# Comparing peformance of TF pretrained model

In this notebook, I am going to train a sample of provided data on every pretrained model provided by tensorflow. following models will be compared
* DenseNet169
* InceptionV3
* MobileNetV2
* NASNetLarge
* ResNet101
* VGG19
* Xception
* EfficientNetB4

This notebook generate a image of plots of loss and accuracy for every model you can download that from output tab

This is not a programming tutorial notebook. Main foucs is on comparing different architectures

Let's set our directories here

In [ ]:
image_directory = "/kaggle/input/cassava-leaf-disease-classification/train_images"
train_csv_dir = "/kaggle/input/cassava-leaf-disease-classification/train.csv"

Following cell loads the train.csv

In [ ]:
import pandas as pd

train_csv = pd.read_csv(train_csv_dir,dtype=str)

1. A useful function which splits data into train, test ensuring each set get equal not of examples for a class. In short it splits data eqaually.
2. Secondly, it take a sample from whole data. where parameter samples refer to no of total examples to consider for each class which then later get split into train and test

In [ ]:
def balanced_train_test_split(df, label_col, samples, split=0.25):
    dfs_train = []
    dfs_test = []

    no_of_classes = len(set(df[label_col].tolist()))
    for class_id in range(0, no_of_classes):
        class_df = df[(df[label_col]==str(class_id))].head(n=samples)
        class_df_train = class_df.sample(frac=1-split)
        class_df_test = class_df.drop(class_df_train.index)
        dfs_train.append(class_df_train)
        dfs_test.append(class_df_test)
    
    
    dfs_train = pd.concat(dfs_train).sample(frac=1)
    dfs_test = pd.concat(dfs_test).sample(frac=1)
    
    return dfs_train, dfs_test

* to save time I will load only 500 examples of each class and use them to train all the networks

In [ ]:
train, test = balanced_train_test_split(train_csv, "label", samples = 500, split=0.2)
train_examples = train.count()[0]
test_examples = test.count()[0]

print("Total training examples: ", train_examples)
print("Total testing examples: ", test_examples)

Following cell makes generator to load images in batches. I am using batch of 16 for each  model and target image size will be 256x256

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size=(256,256)
image_shape=(256,256,3)
batch_size=16

kwargs = {
    "directory":image_directory, "x_col":'image_id', "y_col":'label',
    "target_size":image_size, "color_mode":'rgb', "classes":None,
    "class_mode":'categorical', "batch_size":batch_size, "shuffle":True, 
}

generator = ImageDataGenerator()

train_generator = generator.flow_from_dataframe(train, **kwargs)
test_generator = generator.flow_from_dataframe(test, **kwargs)

Following cell define setting to use for all the models and a function to attach final dense layers to make predictions. I will train each model for 30 epochs with learning rate of 0.001

In [ ]:
from tensorflow.keras.layers import Dropout,Flatten,Dense,BatchNormalization,Activation
import tensorflow as tf

modelsettings = {
    "include_top":False, "weights":'imagenet', "input_shape":image_shape
}

optimize_adam =  tf.keras.optimizers.Adam(learning_rate=0.001)
loss_func = "categorical_crossentropy"
acc_mat = "categorical_accuracy"

fit_settings={
    "x":train_generator, "steps_per_epoch":train_examples//batch_size,
    "validation_data":test_generator, "validation_steps":test_examples//batch_size, 
    "epochs":20, "verbose":1,"shuffle":True
}

def create_model(model):
    model = tf.keras.Sequential([
            model,
            Flatten(),
            Dense(256),
            BatchNormalization(),
            Activation("relu"),
            Dropout(0.5),
            Dense(32),
            BatchNormalization(),
            Activation("relu"),
            Dense(5, activation="softmax"),
        ])
    return model

# Create first model - DenseNet169
in the next cell densenet will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import DenseNet169

DenseNet = create_model(DenseNet169(**modelsettings))

DenseNet.summary()

In [ ]:
DenseNet.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
densenet_history = DenseNet.fit(**fit_settings)

# Create second model - InceptionV3
in the next cell inception will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import InceptionV3

Inception = create_model(InceptionV3(**modelsettings))

Inception.summary()

In [ ]:
Inception.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
inception_history = Inception.fit(**fit_settings)

# Create third model - MobileNetV2
in the next cell mobilenet will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import MobileNetV2

mobilenet = create_model(MobileNetV2(**modelsettings))

mobilenet.summary()

In [ ]:
mobilenet.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
mobilenet_history = mobilenet.fit(**fit_settings)

# Create fourth model - NASNetMobile
in the next cell nasnet will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import NASNetMobile

nasnet = create_model(NASNetMobile(input_shape=(256,256,3), include_top=False, weights=None))

nasnet.summary()

In [ ]:
nasnet.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
nasnet_history = nasnet.fit(**fit_settings)

# Create Fifth model - ResNet101V2
in the next cell resnet will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import ResNet101V2

resnet = create_model(ResNet101V2(**modelsettings))

resnet.summary()

In [ ]:
resnet.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
resnet_history = resnet.fit(**fit_settings)

# Create sixth model - VGG19
in the next cell vgg will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import VGG19

vgg = create_model(VGG19(**modelsettings))

vgg.summary()

In [ ]:
vgg.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
vgg_history = vgg.fit(**fit_settings)

# Create seventh model - xception
in the next cell xception will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import Xception

xception = create_model(Xception(**modelsettings))

xception.summary()

In [ ]:
xception.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
xception_history = xception.fit(**fit_settings)

# Create eighth model - EfficientNet
in the next cell efficientnet will be trained for 20 epochs as per settings above

In [ ]:
from tensorflow.keras.applications import EfficientNetB4

EfficientNet = create_model(EfficientNetB4(**modelsettings))

EfficientNet.summary()

In [ ]:
EfficientNet.compile(optimizer=optimize_adam,loss=loss_func,metrics=[acc_mat])
efficientnet_history = EfficientNet.fit(**fit_settings)

# Plot the results 

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(ax, history,title):
    ax.plot(history.history["loss"], label = "loss")
    ax.plot(history.history["val_loss"], label="val_loss")
    ax.set(xlabel="epochs", ylabel="loss")
    ax.set_title(title)
    ax.legend()
    
def plot_metrics(ax, history,title, metrics):
    ax.plot(history.history[metrics], label=metrics)
    ax.plot(history.history["val_"+metrics], label="val_"+metrics)
    ax.set(xlabel="epochs", ylabel=title)
    ax.set_title(title)
    ax.legend()
    
    
fig, axes = plt.subplots(8, 2, figsize=(18,40))
plt.subplots_adjust(hspace=0.5)

# model 1 - DenseNet169
plot_loss(axes[0,0], densenet_history, "DenseNet169 loss")
plot_metrics(axes[0,1], densenet_history, "DenseNet169 accuracy", "categorical_accuracy")

# model 2 - inceptionv3
plot_loss(axes[1,0], inception_history, "InceptionV3 loss")
plot_metrics(axes[1,1], inception_history, "InceptionV3 accuracy", "categorical_accuracy")

# model 3 - mobilenet
plot_loss(axes[2,0], mobilenet_history, "MobileNetV2 loss")
plot_metrics(axes[2,1], mobilenet_history, "MobileNetV2 accuracy", "categorical_accuracy")

# model 4 - NASnetlarge
plot_loss(axes[3,0], nasnet_history, "NASNetLarge loss")
plot_metrics(axes[3,1], nasnet_history, "NASNetLarge accuracy", "categorical_accuracy")

# model 5 - ResNet101
plot_loss(axes[4,0], resnet_history, "ResNet101V2 loss")
plot_metrics(axes[4,1], resnet_history, "ResNet101V2 accuracy", "categorical_accuracy")

# model 6 - VGG19
plot_loss(axes[5,0], vgg_history, "VGG19 loss")
plot_metrics(axes[5,1], vgg_history, "VGG19 accuracy", "categorical_accuracy")

# model 7 - Xception
plot_loss(axes[6,0], xception_history, "Xception loss")
plot_metrics(axes[6,1], xception_history, "Xception accuracy", "categorical_accuracy")

# model 8 - EfficientNet
plot_loss(axes[7,0], efficientnet_history, "EfficientNetB4 loss")
plot_metrics(axes[7,1], efficientnet_history, "EfficientNetB4 accuracy", "categorical_accuracy")

fig.savefig("Comparingmodels.png", dpi=300)